In [8]:
import pandas as pd
import glob
from fuzzywuzzy import fuzz

In [9]:
# Define column widths
colspecs = [(2, 34), (35, 36), (36, 37), (37, 42)]

# Read the file
sc = pd.read_fwf('../datasets/street_dict.txt', colspecs=colspecs, header=None, names=["address_name", "f", "borough", "street_code"], dtype='string')

In [10]:
# Mappin based on documentation from here: https://data.cityofnewyork.us/City-Government/Street-Name-Dictionary/w4v2-rv6b/about_data
sc.loc[sc.borough == "1", "borough"] = 'NY'
sc.loc[sc.borough == "2", "borough"] = 'BX'
sc.loc[sc.borough == "3", "borough"] = 'K'
sc.loc[sc.borough == "4", "borough"] = 'Q'
sc.loc[sc.borough == "5", "borough"] = 'R'

In [12]:
sc[sc.street_code == '25390'].head()

,address_name,f,borough,street_code
1963,MADISON AVENUE,F,NY,25390
12088,191 STREET,F,Q,25390
17716,CURTIS AVENUE,F,R,25390


In [14]:
base_path = '../datasets/hs/'
years = ['2021', '2016', '2017', '2018', '2019', '2020']
files = [file for year in years for file in glob.glob(base_path + year + ".csv")]

def fuzzy_match(row, threshold=97):  # increase the threshold for a near-perfect match
    return fuzz.token_sort_ratio(row['primary_address_line_1'], row['address_name']) > threshold

for file in files:
    hs = pd.read_csv(file) 

    hs = hs.drop(columns=hs.filter(regex='prgdesc\d*').columns)
    hs = hs.drop(columns=hs.filter(regex='requirement\d*').columns)
    hs = hs.drop(columns=hs.filter(regex='eligibility\d*').columns)
    hs = hs.drop(columns=hs.filter(regex='auditioninformation\d*').columns)
    hs = hs.drop(columns=hs.filter(regex='admissionspriority\d*').columns)


    hs.loc[hs.boro == "M", "boro"] = 'NY'
    hs.loc[hs.boro == "X", "boro"] = 'BX'

    hs.loc[hs.boro == "Manhattan", "boro"] = 'NY'
    hs.loc[hs.boro == "Bronx", "boro"] = 'BX'
    hs.loc[hs.boro == "Queens", "boro"] = 'Q'
    hs.loc[hs.boro == "Staten Island", "boro"] = 'R'
    hs.loc[hs.boro == "Brooklyn", "boro"] = 'K'

    # Convert the columns to lowercase to make the matching easier
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.lower()
    sc['address_name'] = sc['address_name'].str.lower()

    # Remove extra whitespaces between words in 'primary_address_line_1' and 'address_name'
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('\s+', ' ', regex=True)
    sc['address_name'] = sc['address_name'].str.replace('\s+', ' ', regex=True)

    # Remove suffixes like 'th', 'st', 'nd', 'rd' from 'primary_address_line_1'
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('(\d+)(st|nd|rd|th)', r'\1', regex=True)
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('st.', 'st')
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('ave.', 'avenue')
    
    # First, join on borough column
    merged = sc.merge(hs, left_on='borough', right_on='boro', suffixes=('', '_y'))

    merged['primary_address_line_1'] = merged['primary_address_line_1'].str.replace('^\S+\s*', '', regex=True)

    # Fuzzy match entry addreses. Not needed anymore. I used it to see what are the difference in the street so I can
    # add the rules above
    mask = merged.apply(fuzzy_match, axis=1)

    temp = merged[mask]
    diff = temp[temp.address_name != temp.primary_address_line_1]
    print(f"Year: {file} \n New: {len(temp)} \n Old: {len(hs)}")
    print("------------------------")

    missing_schools = hs[~hs['school_name'].isin(temp['school_name'])]

    if '2021' in file:
        merged[mask].to_csv('../datasets//hs/combined_hs_with_street_codes.csv', mode='w', header=True, index=False)

    merged[mask].to_csv('../datasets//hs/combined_hs_with_street_codes.csv', mode='a', header=False, index=False)

Year: ../hs/2021.csv 
 New: 421 
 Old: 442
------------------------


TypeError: __cinit__() got an unexpected keyword argument 'mode'

In [18]:
hssc = pd.read_csv("../hs/combined_hs_with_street_codes.csv", low_memory=False)

hssc.to_parquet("../hs/combined_hs_with_street_codes.parquet", index=False)